<a href="https://colab.research.google.com/github/schaidez2727/MAT-421/blob/main/ModuleC_MAT421_Chaidez.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Module C (HW 4) - MAT 421 #
#### Santana Chaidez ####

____
## Root Finding Problem Statement ##
___

* **Root**, or **zero**, of a function == x_r such that f(x_r) = 0
* Finding the roots of a function aid in many engineering applications, such as signal processing and process optimization.
* Root finding is a simple process for quadratic functions, but can be difficult for more complex/higher-order functions.
* Can be done analytically, or using numerical approximations.

Let's do an example of numerical approximation for root finding, and then verify that the solution is a root:

In [21]:
# An example of using numerical approximation to find the root of the function cos(x) - 2x near -1

import numpy as np
# importing Python library, NumPy, which fascilitates various forms of scientific computing
from scipy import optimize
# importing SciPy library so we can use fsolve function to compute the root of a function near a value

f = lambda x: np.cos(x) - 2*x # define the function
x_r = optimize.fsolve(f, -1) # approximate the root
print("x_r =", x_r)

f_r = f(x_r) # verify the root value approximation
soln = np.round(f_r, 10)
print("f(x_r) = ", soln)

x_r = [0.45018361]
f(x_r) =  [-0.]


The example shows that the root was successfully approximated. Before rounding, the solution produced by the function by inputting the root is a very small value, -3.1086e-15. So it is very close to zero, as seen with the high amount of digits we rounded to.

Now let's test fsolve on a function that has no roots: f(x) = 1/x^(1/2)

In [22]:
# Using numerical approximation on a function with no roots

f = lambda x: 1/x**(1/2) # define the function
x_r, infodict, ier, mesg = optimize.fsolve(f, 10, full_output=True) # approximate the root, set full_output to True
print("x_r = ", x_r)

result = f(x_r) # verify the result
print("f(x_r) = ", result)

print(mesg) # show full_output message

x_r =  [8.81854734e+120]
f(x_r) =  [3.36745247e-61]
The number of calls to function has reached maxfev = 400.


As we can see, because we tirned on full_output, we recieved information about out output in the message. A message is returned if no solution is reached, and we recieve this message telling us that a solution was not being found in a reasonable amount of iterations.

____
## Tolerance ##
___

* **Tolerance** is the level of error deemed acceptable in an engineering application, where **error** is the deviation from an expected/computed value.
* Once a computer program has found a solution with an error smaller than the set tolerance, it has **converged** to a solution.
* Choosing how to measure the error and setting the tolerance must be done with careful consideration of the applicational purposes and nuances of the programs being used in order to ensure appropriate results.

Let's practice setting tolerance and calculating error using the function with no roots, 1/x:

In [23]:
# Using tolerance and error on numerical approximation ona function with no roots

f = lambda x: 1/x # define the function

x_r, infodict, ier, mesg = optimize.fsolve(f, -2, xtol=.1, full_output=True)
# approximate the root, turn on full_output, set tolerance to 0.1
print("x_r =", x_r)

error = f(x_r) # calculating error as distance of solution from zero
print("error value", error)

print(mesg) # show full_output message

x_r = [-3.52047359e+83]
error value [-2.84052692e-84]
The number of calls to function has reached maxfev = 400.


As we can see, the program has attempted to converge to a very large number as the root, due to the function 1/x being asymptotic to the x-axis as the function approached f(x) = 0.

____
## Bisection Method ##
___

* The **bisection method** uses the **intermediate value theorem** to find roots.
* **Intermediate value theorem** (IVT): if f(x) is a continuous function between *a* and *b*, and f(a) =/= f(b), then there must be a value *c* such that *a* < *c* < *b* and f(c) = 0.
* For the bisection method, *a* and *b* are real scalar values such that *a* < *b*. Assume f(a) > 0 and f(b) < 0.
* The bisection method calculates the midpoint between these these two points, f(m = (b+a)/2).
* If f(m) is within tolerance and converges to zero, m is declared the root.
* If f(m) is outside tolerance and positive, it becomes the new left bound, a.
If f(m) is outside the tolerance and negative, it becomes the new right bound, b.

Let's set up a function to carry out this process.

In [24]:
def my_bisection(f, a, b, tol):
    # approximates a root, r, of f bounded
    # by a and b to within tolerance
    # | f(m) | < tol with the midpoint
    # m between a and b

    # check if a and b bound a root
    if np.sign(f(a)) == np.sign(f(b)):
        raise Exception(
         "The scalar values a and b do not bound a root")

    # calculate midpoint b/w bounds a and b
    m = (a + b)/2

    if np.abs(f(m)) < tol:
        # stopping condition, report m as root
        return m
    elif np.sign(f(a)) == np.sign(f(m)):
        # case where m is an improvement on left bound, a
        # make recursive call with a = m
        return my_bisection(f, m, b, tol)
    elif np.sign(f(b)) == np.sign(f(m)):
        # case where m is an improvement on right bound, b
        # make recursive call with b = m
        return my_bisection(f, a, m, tol)

In [25]:
# Use bisection method function on function f(x) = x^2 - 1

f = lambda x: x**2 - 2 # define function f

r1 = my_bisection(f, 0, 2, 0.1) # performing bisection w/ higher tolerance, 0.1
print("r1 =", r1)
r01 = my_bisection(f, 0, 2, 0.01) # performing bisection w/ lower tolerance, 0.01
print("r01 =", r01)

print("f(r1) =", f(r1))
print("f(r01) =", f(r01))

r1 = 1.4375
r01 = 1.4140625
f(r1) = 0.06640625
f(r01) = -0.00042724609375


As we can see, both approximated roots do produce a value close to zero, but the bisection with a lower set tolerance produces a mmore accurate root/solution than the bisection with a higher tolerance.

____
## Newton-Raphson Method ##
___

* The **Newton-Raphson method** finds the unknown root x_r of a continuous funnction f(x) by beginning with a guess for the root, x_0.
* Taking the linear approximation of f(x) around x_0, we can find the intersection of this line with the x-axis. This becomes our improved guess, x_1:

f(x) ~= f(x_0) + f'(x_0)(x - x_0); f(x_1) = 0 --> x_1 = x_0 - f(x_0)/f'(x_0)
* This is a **Newton step**.
* The Newton-Raphson method iterates Newton steps from x_0 until the error is less than the tolerance.

Let's try this method on the function f(x) = x^2 - 1, which has a zero of 1^(1/2) = 1:

In [26]:
# Use Newton-Raphson method on function f(x) = x^2 - 1

f = lambda x: x**2 - 1 # define function f(x)
f_prime = lambda x: 2*x # define derivative f'(x) as slope for linear approximation
newton_raphson = 1.4 - (f(1.4))/(f_prime(1.4)) # perform Newton step to find new root guess

print("newton_raphson = ", newton_raphson)
print("sqrt(1) = ", np.sqrt(1)) # compare Newton-Raphson value with known root value

newton_raphson =  1.0571428571428572
sqrt(1) =  1.0


Define a function to perform Newton-Raphson method iteratively until solution is within a specified tolerance:

In [27]:
def my_newton(f, df, x0, tol):
    # output is an estimation of the root of f using the Newton Raphson method
    # recursive implementation
    if abs(f(x0)) < tol: # calculating error as |f(x0)| and checking if it is w/in tolerance
        return x0 # if error is w/in tolerance, return root guess as converged approximation
    else:
        return my_newton(f, df, x0 - f(x0)/df(x0), tol)
        # if not w/in tolerance, perform another iteration w/ a new guess
        # by performing another Newton step

In [28]:
# Using Newton-Raphson function to compute root for f(x) = x^2 - 1.5
# to within a tolerance of 1e-6, with initial guess of x_0 = 1.5

f = lambda x: x**2 - 1.5 # define function f(x)
f_prime = lambda x: 2*x # define derivative f'(x) as slope for linear approximation

estimate = my_newton(f, f_prime, 1.5, 1e-6) # input f(x), f'(x), x0, and tolerance into function
print("estimate = ", estimate) #display estimate from function
print("sqrt(1.5) = ", np.sqrt(1.5)) # compare with know root value

estimate =  1.2247448979591837
sqrt(1.5) =  1.224744871391589


The function successfully converged to a solution that is very close to the actual root value!

____
## Root Finding in Python ##
___

Let us briefly test out one of Python's existing root-finding functions from the SciPy library, *fsolve*, to find the roots of a higher-order polynomial function.

In [29]:
from scipy.optimize import fsolve
# importing existing root-finding function, fsolve

# Use fsolve on cubic polynomial f(x) = x^3 - 75x^2 - 20x + 10
# which has two roots: x = -0.51965 and x = 0.25568

f = lambda x: x**3 - 75*x**2 - 20*x +10

fsolve(f, [-1, 1]) # setting arguments for initial guesses for roots, x0 = [-1, 1]

array([-0.51964943,  0.25568335])

As we can see, it did find the roots. But this function works best when the number of roots are known and the guesses are appropriately close to the actual roots.